In [3]:
import cv2
import mediapipe as mp
import time

# 初始化MediaPipe人脸检测
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 设置人脸检测器
# min_detection_confidence: 检测置信度阈值，降低可以提高检测率但可能增加误检
face_detection = mp_face_detection.FaceDetection(
    model_selection=1,  # 0: 近距离人脸 (2米内), 1: 远距离人脸 (5米内)
    min_detection_confidence=0.3
)

# 打开摄像头
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # 降低分辨率以提高FPS
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# FPS计算
fps_time = 0
fps_counter = 0
fps_display = 0

print("按 'q' 退出程序")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("无法读取摄像头画面")
        break
    
    # 开始计时
    start_time = time.time()
    
    # 将BGR图像转换为RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 处理图像，检测人脸
    results = face_detection.process(rgb_frame)
    
    # 绘制人脸检测结果
    if results.detections:
        for detection in results.detections:
            # 获取边界框
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            
            # 转换为像素坐标
            x = int(bbox.xmin * w)
            y = int(bbox.ymin * h)
            width = int(bbox.width * w)
            height = int(bbox.height * h)
            
            # 绘制边界框
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            
            # 显示置信度
            confidence = detection.score[0]
            cv2.putText(frame, f'{confidence:.2f}', (x, y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # 绘制关键点（可选）
            # mp_drawing.draw_detection(frame, detection)
    
    # 计算并显示FPS
    fps_counter += 1
    if time.time() - fps_time > 1:
        fps_display = fps_counter
        fps_counter = 0
        fps_time = time.time()
    
    cv2.putText(frame, f'FPS: {fps_display}', (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    
    # 显示结果
    cv2.imshow('Face Tracking - MediaPipe', frame)
    
    # 按 'q' 退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
cap.release()
cv2.destroyAllWindows()
face_detection.close()

按 'q' 退出程序
